<a href="https://colab.research.google.com/github/DCDPUAEM/DCDP/blob/main/04%20Deep%20Learning/notebooks/08-Embeddings-imagenes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Embeddings de Imágenes</h1>

En esta notebook exploraremos dos modelos pre-entrenados de embeddings de imagenes basado en arquitecturas CNN.

El primer modelo está basado en [CLIP](https://github.com/openai/CLIP), es una implementación que no necesita Pytorch ni Tensorflow y ya está especializada en la obtención de embeddings ([documentación](https://github.com/minimaxir/imgbeddings)).

El segundo modelo es un autoencoder convolucional implementado y entrenado en una de las sesiones anteriores ([notebook](https://github.com/DCDPUAEM/DCDP/blob/main/04%20Deep%20Learning/notebooks/06-Autoencoders.ipynb)). Leeremos el modelo guardado y entrenado de esa ocasión.

Con estos embeddings realizaremos dos tareas:

* Obtención de vecinos más cercanos.
* Extracción de features para una tarea de clasificación.

Primero, instalamos el módulo para el primer modulo.

In [ ]:
!pip3 install -qq imgbeddings

## Example

Obtengamos el embedding de una imagen de prueba

In [ ]:
import requests
from PIL import Image

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.imshow(image)
plt.axis('off')
plt.show()

In [ ]:
from imgbeddings import imgbeddings

embeddings_model = imgbeddings()

Generamos el embedding de esta imágen y mostramos sus primeras 5 componentes

In [ ]:
embedding = embeddings_model.to_embeddings(image)
print(embedding[0][:5])
print(embedding[0].shape)

# Vecinos más cercanos usando Fashion-MNIST

Creamos una instancia del modelo de embeddings pre-entrando

In [ ]:
from imgbeddings import imgbeddings

embeddings_model = imgbeddings()

Cargamos el dataset Fashion-MNIST y re-escalamos

In [ ]:
from keras.datasets import fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0

Generaremos los embeddings de unas las primeras 10 imágenes. Generar los embeddings de todo el conjunto de entrenamiento o prueba tarda alrededor de 3 horas, por esa razón no lo haremos en esta notebook.

In [ ]:
import numpy as np
from PIL import Image

x_sample = x_train[:10]

images = [Image.fromarray(x) for x in x_sample]
embeddings_sample = embeddings_model.to_embeddings(images)
embeddings_sample.shape

Descargaremos los embeddings de todo el conjunto de entrenamiento y prueba, fueron generados previamente de la misma manera mostrada en la celda anterior.

In [ ]:
!pip install -qq gdown

In [ ]:
# ---- FASHION MNIST ----

!gdown 13hmH7mgvGrYYyk0QoWyTGDxv2L3BtfmA # train
!gdown 1t5_qj0YryhNKLMuAVoLlALBmvnMT8L3M # test

Escojamos el conjunto de entrenamiento o prueba.

In [ ]:
import numpy as np

# X = x_train.copy()
# y = y_train.copy()
# embeddings = np.load("Fmnist_train_imgbeddings.npy")

X = x_test.copy()
y = y_test.copy()
embeddings = np.load("Fmnist_test_imgbeddings.npy")

embeddings.shape

Entrenmos una instancia de [`NearestNeighbors`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html) para encontrar de forma **eficiente** los vecinos más cercanos de cualquier embedding. El uso de la métrica `cosine` es muy típico para los embeddings generados por modelos de aprendizaje profundo.

In [ ]:
from sklearn.neighbors import NearestNeighbors

num_neighbors = 3

nns = NearestNeighbors(metric='cosine')
nns.fit(embeddings)

Selecciones 4 imágenes al asar del dataset y de cada una, encontremos los 3 vecinos más cercanos. ¿Qué tan buenos son los resultados?

In [ ]:
import matplotlib.pyplot as plt

num_to_check = 4

idxs_to_check = np.random.choice(list(range(X.shape[0])), num_to_check)

distances, NN = nns.kneighbors(embeddings[idxs_to_check,:],
                               num_neighbors+1,
                               return_distance=True)

print(f"Índices de los vecinos más cercanos:\n{NN}")
print(f"Distancia a los vecinos más cercanos:\n{distances}")

distances = distances[:,1:]
NN = NN[:,1:]

fig, axs = plt.subplots(num_to_check,num_neighbors+1, figsize=(4*num_neighbors, 4*num_to_check))
for k,idx in enumerate(idxs_to_check):
    axs[0,k].imshow(X[idx])
    axs[0,k].axis('off')
    for i,nn_idx in enumerate(NN[k][:num_neighbors]):
        axs[i+1,k].imshow(X[nn_idx])
        axs[i+1,k].set_title(f"NN {i+1},\ndistance:{np.round(distances[k,i],3)}")
        axs[i+1,k].axis('off')
fig.show()

# Features para la clasificación Cats vs Dogs

En una notebook de redes CNN realizamos la tarea de clasificación del dataset *Cats vs Dogs*. Logramos obtener un accuracy del orden de 80%.

Usaremos embeddings generados con un autoencoder CNN como features para entrenar un clasificador de Machine Learning *clásico* y ver cómo se compara con el clasificador CNN antes mencionado.

Descargemos los modelos ya entrenados del autoencoder CNN

In [ ]:
!gdown 1adFS_RZK9b20rTKZlC4fTDuZtOEy9np1    # ENCODER
!gdown 1-0Y_aV5XST4GnySszgnIi5AC6xoQOdiG    # DECODER
!gdown 1Ce3u8dwYYriLkz5OpcGn72xIQENIHZX5    # DATASET CATS VS DOGS, versión reducida

In [ ]:
from zipfile import ZipFile

file_name = '/content/cnn_perros_gatos-small.zip'

with ZipFile(file_name, 'r') as myzip:
    myzip.extractall()
    print('Listo')

Definimos los generadores de imágenes. Observa el `batch_size`.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_dir = 'cnn_perros_gatos/train'
test_dir = 'cnn_perros_gatos/test'

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(144, 144),
        batch_size=1,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(144, 144),
        batch_size=1,
        class_mode='binary')

Leemos los modelos encoder y decoder ya entrenados.

🔵 ¿Por qué tenemos el WARNING que nos muestra?

In [ ]:
from keras.saving import load_model

CvD_encoder = load_model("perros_gatos_encoder_cnn_30epochs.h5")
CvD_decoder = load_model("perros_gatos_decoder_cnn_30epochs.h5")

In [ ]:
CvD_encoder.summary()

In [ ]:
CvD_decoder.summary()

## Clasificación

Obtenemos las features y etiqueta de clase de cada imagen. Esto lo hacemos iterando sobre el generador de imágenes, recuerda que tenemos que detener la iteración manualmente, de otra forma la iteración nunca termina.

En este paso es que es importante el `batch_size` definido en el paso anterior.

La ejecución dura alrededor de 6 minutos.

In [ ]:
import numpy as np

train_codes = np.zeros((2000, 512))
y_train = np.zeros((2000,))

train_tensors = np.zeros((2000, 144, 144, 3))
test_tensors = np.zeros((1000, 144, 144, 3))

test_codes = np.zeros((1000, 512))
y_test = np.zeros((1000,))

for k,(xi,yi) in enumerate(train_generator):
    if k < 2000:
        train_codes[k,:] = CvD_encoder.predict(xi,verbose=0)
        y_train[k] = yi[0]
        train_tensors[k] = xi[0]
    else:
        break

for k,(xi,yi) in enumerate(test_generator):
    if k < 1000:
        test_codes[k,:] = CvD_encoder.predict(xi,verbose=0)
        y_test[k] = yi[0]
        test_tensors[k] = xi[0]
    else:
        break

print(train_codes.shape)
print(y_train.shape)
print(test_codes.shape)
print(y_test.shape)

A partir de este punto, el proceso es el mismo del Machine Learning *clásico*

Veamos el rendimiento de varios algoritmos de Machine Learning

In [ ]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

clfs = [SVC(), DecisionTreeClassifier(), RandomForestClassifier(), KNeighborsClassifier(), LogisticRegression()]
names = ["SVC", "DecisionTreeClassifier", "RandomForestClassifier", "KNeighborsClassifier", "LogisticRegression"]

for clf,name in zip(clfs,names):
    clf.fit(train_codes, y_train)
    print(f"{name}: {clf.score(test_codes, y_test)}")

## Vecinos más cercanos

Descargamos una imagen de prueba

In [ ]:
!gdown 1ofrBZ-E9JO3nAfAYKN0l2B6rsUbXwOVp

La preprocesamos

In [ ]:
from keras import utils

img_path = '/content/cat.png'
test_image = utils.load_img(img_path, target_size=(144, 144))
test_image = utils.img_to_array(test_image)
test_image = test_image.reshape(1,144,144,3)
test_image /= 255.
test_image.shape

La mostramos

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.imshow(test_image[0])
plt.axis('Off')
plt.show()

Generamos el embedding de la imagen de arriba, es decir, obtenemos su codificación latente mediante el codificador pre-entrenado.

In [ ]:
test_image_code = CvD_encoder.predict(test_image)
print(test_image_code.shape)
test_image_code = test_image_code.reshape(512,) # Hacemos el reshape adecuado
test_image_code.shape

Incializamos y entrenamos una instancia de `NearestNeighbors`, una vez más usando la métrica de la similitud coseno.

In [ ]:
from sklearn.neighbors import NearestNeighbors

num_neighbors = 3
nns = NearestNeighbors(metric='cosine')
nns.fit(train_codes)

Mostramos las 3 imágenes más similares a la imágen de muestra, junto con sus distancias en el espacio de codificación latente.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


distances, NN = nns.kneighbors([test_image_code],
                               num_neighbors+1,
                               return_distance=True)

distances = distances[:,:-1]
NN = NN[:,:-1]

print(f"Índices de los vecinos más cercanos:\n{NN}")
print(f"Distancia a los vecinos más cercanos:\n{distances}")

fig, axs = plt.subplots(1,num_neighbors+1, figsize=(4*num_neighbors, 4))
axs[0].imshow(test_image[0])
axs[0].axis('off')
for i,nn_idx in enumerate(NN[0]):
    axs[i+1].imshow(train_tensors[nn_idx])
    axs[i+1].set_title(f"NN {i+1},\ndistance:{round(distances[0,i],2)}")
    axs[i+1].axis('off')
fig.show()

🔵 Hay algunas imágenes no tan parecidas, ¿qué factores pueden afectar el rendimiento de esta tarea?